In [2]:
!pip3 install apache_beam

     |████████████████████████████████| 10.1 MB 5.3 MB/s 
     |████████████████████████████████| 508 kB 68.9 MB/s 
     |████████████████████████████████| 255 kB 52.6 MB/s 
     |████████████████████████████████| 2.3 MB 40.7 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 151 kB 29.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 21.4 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=9c6b42d0ea57d1c910d0a87499bccea81088c5cec5cb5138e64afb7ef6f22fcc
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
Successfully built dill
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling

In [1]:
import apache_beam as beam

In [3]:
!ls

sample_data  students.txt


## Map 

•	Applies a simple 1-to-1 mapping function over each element in the collection.

In [13]:
def strip_header_and_newline(text):
  return text.strip('# \n')

with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          '# 🍓Strawberry\n',
          '# 🥕Carrot\n',
          '# 🍆Eggplant\n',
          '# 🍅Tomato\n',
          '# 🥔Potato\n',
      ])
      | 'Strip header' >> beam.Map(strip_header_and_newline)
      | beam.Map(print))

🍓Strawberry
🥕Carrot
🍆Eggplant
🍅Tomato
🥔Potato


**MapTuple** for key-value pairs. If your PCollection consists of (key, value) pairs, you can use MapTuple to unpack them into different function arguments.

In [ ]:
# MapTuple for key-value pairs
with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
      ])
      | 'Format' >> beam.MapTuple(lambda icon, plant: '{}{}'.format(icon, plant))
      | beam.Map(print))

## FlatMap

•	Applies a simple 1-to-many mapping function over each element in the collection. The many elements are flattened into the resulting collection.

In [ ]:
def split_words(text):
  return text.split(',')

with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          '🍓Strawberry,🥕Carrot,🍆Eggplant',
          '🍅Tomato,🥔Potato',
      ])
      | 'Split words' >> beam.FlatMap(split_words)
      | beam.Map(print))

**FlatMapTuple for key-value pairs**

If your PCollection consists of (key, value) pairs, you can use FlatMapTuple to unpack them into different function arguments.

In [ ]:
def format_plant(icon, plant):
  if icon:
    yield '{}{}'.format(icon, plant)

with beam.Pipeline() as pipeline:
  plants = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          ('🍓', 'Strawberry'),
          ('🥕', 'Carrot'),
          ('🍆', 'Eggplant'),
          ('🍅', 'Tomato'),
          ('🥔', 'Potato'),
          (None, 'Invalid'),
      ])
      | 'Format' >> beam.FlatMapTuple(format_plant)
      | beam.Map(print))

## Filter

Given a predicate, filter out all elements that don’t satisfy that predicate. May also be used to filter based on an inequality with a given value based on the comparison ordering of the element.

In [ ]:
def is_perennial(plant):
  return plant['duration'] == 'perennial'

with beam.Pipeline() as pipeline:
  perennials = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'
          },
      ])
      | 'Filter perennials' >> beam.Filter(is_perennial)
      | beam.Map(print))

In [ ]:
#Filtering with multiple arguments

def has_duration(plant, duration):
  return plant['duration'] == duration

with beam.Pipeline() as pipeline:
  perennials = (
      pipeline
      | 'Gardening plants' >> beam.Create([
          {
              'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'
          },
          {
              'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'
          },
          {
              'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'
          },
          {
              'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'
          },
          {
              'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'
          },
      ])
      | 'Filter perennials' >> beam.Filter(has_duration, 'perennial')
      | beam.Map(print))

In [5]:
with beam.Pipeline() as pipeline:
  students = (
      pipeline
      |"Read from text" >> beam.io.ReadFromText("students.txt", skip_header_lines= True)
      |"spliting the record" >> beam.Map(lambda record : record.split(','))
      |"filtering the data with PASS" >> beam.Filter(lambda record : record[5]=="FAIL")
      |"Write to text" >> beam.io.WriteToText("result/pass_students")
  )


In [3]:
!ls

result	sample_data  students.txt


In [6]:
!{('head -n 10 result/pass_students-00000-of-00001')}

['1', 'vignesh', 'chn', '27', '15', 'FAIL']
['2', 'joey', 'us', '51', '20', 'FAIL']
['6', 'sree', 'koc', '25', '27', 'FAIL']
['9', 'tinkle', 'ker', '27', '9', 'FAIL']
